In [11]:
import json
import datetime
import time
import urllib2
import nbimporter
import random
import os
import psutil
import psycopg2
import Insert_py
import Get_py
import Tor
import Proxy_Gatherer as pg
import Init_database as indb
import warnings
warnings.filterwarnings("ignore")

In [2]:
#--------INITIALIZING CONNECTION TO DATABASE--------#
db_name="db_teld"
user_name="postgres"
password = "postgres"
host="127.0.0.1"
port="5432"
conn = indb.init_db(db_name, user_name, password, host, port)

In [3]:
#------- Gathering proxies for visiting teld.cn -------##
def proxy():
    hdr = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36',
           'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8'}
    gatherproxy = pg.ProxyGatherer(hdr)
    proxies_xici = gatherproxy.getproxies_xici()
    proxies_elite = gatherproxy.getproxies_elite()
    # t = time.time()
    proxiesXici_pool = gatherproxy.proxies_cleanup(proxies_xici) # Could also specify timeout limit you prefer, 
                                                                    #format--(connect time, read time)
    proxiesElite_pool = gatherproxy.proxies_cleanup(proxies_elite)
    proxies = proxiesXici_pool+proxiesElite_pool
    return proxies

In [4]:
def write_error(error_log, pillcode, error_msg):
    """
        Input:
            error_log - file, write error messages into designated file
    """
    error_log.write(pillcode)
    error_log.write(str(error_msg))
    error_log.write('\n')

In [5]:
def pillState_scrap(pillcode): 
    """
        Input:
            pillCode - string, each charging pile has its own pile code
        Output:
            pill_detailedInfo - dictionary of pill state details
    """
    hdr = {
            'accept': "*/*",
            'user-agent': "Teld/3.3.0 (iPhone; iOS 10.3.3; Scale/2.00)",
            'cookie': "TELDAppID=",
            'content-length': "111",
            'connection': "keep-alive",
            'accept-encoding': "gzip, deflate",
            'device': "app_version=3.3.0&os_version=10.3.3&client=ios&device_name=iPhone 6s (A1633/A1688/A1691/A1700)&device_id=38CD294C-CC36-4998-9074-C7F798EE6087&city_code=4403&city_name=%E6%B7%B1%E5%9C%B3&lat=0.000000&lng=0.000000&network=wifi&location_city_name=",
            'host': "basesg.teld.cn",
            'content-type': "application/x-www-form-urlencoded",
            'cache-control': "no-cache"
        }
    time_out = 50
    F = None
    proxies_pool = None
    #If encounter error, try one more request with a different proxy
    try:
        pill_detailedInfo, proxies_pool = Get_py.get_onePill_APP(hdr, pillcode, F, proxies_pool, time_out=time_out)
    except ValueError as e:
        if F is not None:
            write_error(F, pillcode, e)
        try:
            pill_detailedInfo, proxies_pool = Get_py.get_onePill_APP(hdr, pillcode, F, proxies_pool, time_out=time_out)
        except ValueError as e:
            if F is not None:
                write_error(F, pillcode, e)
            pill_detailedInfo = None
        except (requests.exceptions.ChunkedEncodingError) as e:
            if F is not None:
                write_error(F, pillcode, e)
#             else:
            pill_detailedInfo = None
    except (requests.exceptions.ChunkedEncodingError) as e:
        if F is not None:
            write_error(F, pillcode, e)
        try:
            pill_detailedInfo, proxies_pool = Get_py.get_onePill_APP(hdr, pillcode, F, proxies_pool, time_out=time_out)
        except ValueError as e:
            if F is not None:
                write_error(F, pillcode, e)
            pill_detailedInfo = None
        except (requests.exceptions.ChunkedEncodingError) as e:
            if F is not None:
                write_error(F, pillcode, e)
#         else:
            pill_detailedInfo = None
#     return pill_detailedInfo, proxies_pool
    except:
        pill_detailedInfo = None
    return pill_detailedInfo

In [6]:
def insert_state(outputs):
    for pill_detailedInfo in outputs:
        if pill_detailedInfo is not None:
            db_name="db_teld"
            user_name="postgres"
            password = "postgres"
            host="127.0.0.1"
            port="5432"
            conn = indb.init_db(db_name, user_name, password, host, port)
            pillCode = str(pill_detailedInfo['pillCode'])
            stateCode = str(pill_detailedInfo['stateCode'])
            stateCodeV3 = str(pill_detailedInfo['stateCodeV3'])
            if pill_detailedInfo['stateName'] is not None:
                stateName = str(pill_detailedInfo['stateName'].encode('utf-8'))
            else:
                stateName = ''
            if pill_detailedInfo['stateNameV3'] is not None:
                stateNameV3 = str(pill_detailedInfo['stateNameV3'].encode('utf-8'))
            else:
                stateNameV3 = ''
            conn = Insert_py.insert_state_info(conn, pillCode, stateCode, stateName, stateCodeV3, stateNameV3)
            conn.close()

In [7]:
# #------- Request pilecode from db for states -------#
try:
    cur = conn.cursor()
    sta_list = ['b134a159-16bd-4f5d-af1b-cd42599a3bab', '33a3c031-0a3a-417d-ba1b-53549bb08dc2', 
                'e4758edb-1183-4c51-887b-a52ca7abef3d', '13bdab71-110b-4448-a3c4-5c3113183546']
    #1. beijing hangkong jingmi jixie yanjiusuo chongdianzhan --'2b3a35d4-cb94-4b8f-a47a-bbb054754066',
    #2. beijing xijiao bingguan chongdianzhan -- '1d02640a-7f14-4fc3-baaf-60754a38fff7
    #3. beijing yangguanglicheng jiudian chongdianzhan --'ac6cb946-5fff-491b-859e-55cc5f701657',
    #4. beijing rongtao gongsi chongdianzhan -- '32d317a9-3d0c-4458-8357-89a2b3b1cf94'
    #5. beijing huairou qu yanshui lu -- 'dd4a3ddc-e1b0-4e2e-b7ca-faef627f4e66'
    
    #6. shenzhen hengjiang dasha chongdianzhan -- 'b134a159-16bd-4f5d-af1b-cd42599a3bab'
    #7. shenzhen yinxing gongyeyuan chongdianzhan -- '33a3c031-0a3a-417d-ba1b-53549bb08dc2'
    #8. shenzhe wanke jiulongshan chongdianzhan -- 'e4758edb-1183-4c51-887b-a52ca7abef3d'
    #9. shenzhen jingyuan dasha chongdianzhan -- '13bdab71-110b-4448-a3c4-5c3113183546'
    stmt = "select pilecode from tbl_APP_pilecode where staid in %s"
    cur.execute(stmt, (tuple(sta_list),))
    pillcode_list = cur.fetchall()
    cur.close()
except psycopg2.Error as e:
    print(e)
print(len(pillcode_list))

46


In [8]:
for pillcode in pillcode_list:
    print(pillcode[0])

4403070013101
4403070013102
4403070013105
4403070013111
4403070013107
4403070013106
4403070013108
4403070013110
4403070013112
4403070013109
4403060011105
4403060011103
4403060011109
4403060011201
4403060011106
4403060011203
4403060011202
4403060011108
4403060011210
4403060011102
4403060011107
4403060011110
4403060011207
4403060011209
4403060011101
4403060011104
4403060011206
4403060011205
4403060011204
4403060016102
4403060016101
4403060072102
4403060072110
4403060072112
4403060072104
4403060072106
4403060072107
4403060072113
4403060072111
4403060072103
4403060072108
4403060072109
4403060072101
4403060072115
4403060072105
4403060072114


In [9]:
hdr = {
        'accept': "*/*",
        'user-agent': "Teld/3.3.0 (iPhone; iOS 10.3.3; Scale/2.00)",
        'cookie': "TELDAppID=",
        'content-length': "111",
        'connection': "keep-alive",
        'accept-encoding': "gzip, deflate",
        'device': "app_version=3.3.0&os_version=10.3.3&client=ios&device_name=iPhone 6s (A1633/A1688/A1691/A1700)&device_id=38CD294C-CC36-4998-9074-C7F798EE6087&city_code=4403&city_name=%E6%B7%B1%E5%9C%B3&lat=0.000000&lng=0.000000&network=wifi&location_city_name=",
        'host': "basesg.teld.cn",
        'content-type': "application/x-www-form-urlencoded",
        'cache-control': "no-cache"
        }

In [10]:
proxies = proxy()

Got rid of 19 unusable proxies.
Got rid of 43 unusable proxies.


In [39]:
proxy = proxies[0]
proxy

{'http': 'http://175.11.122.194:80', 'https': 'https://175.11.122.194:80'}

In [128]:
import urllib
proxy = {'https': 'https://175.11.122.194:80'}
pillCode = '4403060011105'
url = "https://basesg.teld.cn/api/invoke"
# querystring = {"SID":"BaseApi-App0304_GetTerminalDetails"}
param = {'SID':'BaseApi-App0304_GetTerminalDetails'}
         #'param': '%7B%0A%20%20%22pillCode%22%20%3A%20%22{}%22%2C%0A%20%20%22plateform%22%20%3A%20%22app%22%0A%7D'.format(pillCode)}
param = urllib.urlencode(param)

proxy_support = urllib2.ProxyHandler(proxy)
opener = urllib2.build_opener(proxy_support, urllib2.HTTPSHandler(debuglevel=1))
urllib2.install_opener(opener)

req = urllib2.Request(url, data=param, headers=hdr)


In [130]:
html = urllib2.urlopen(req).read()

URLError: <urlopen error timed out>

In [129]:
param

'SID=BaseApi-App0304_GetTerminalDetails'